In [1]:
import os.path

import wptools
import pandas as pd
import sys, os

In [2]:
DATA_DIR = '../data/data_processed/' 

In [3]:
old_articles = pd.read_csv(DATA_DIR + 'data_processed.csv', index_col=0)
old_articles.index.names = ['id']

In [4]:
minDate = 1995
articles = old_articles[(old_articles.end_date_clean >= minDate) & (old_articles.categories.str.contains("conflict"))].reset_index().reset_index().rename(columns={'index': 'n_conflict'})

In [5]:
articles

,n_conflict,id,title,categories,article_lenght,views,refs_count,link_count,death,locations,end_date_clean,ongoing
0,0,16760,Kosovo War,military conflict,197476,2368366,274,671,2057,Kosovo Q1231,1999.0,False
1,1,22738,Operation Enduring Freedom,military conflict,52281,1248100,60,228,45000,Afghanistan Q889,2014.0,False
2,2,30770,The Troubles,military conflict,143560,4660829,214,578,1049,Northern Ireland Q26,1998.0,False
3,3,43063,Operation Anaconda,military conflict,53930,428455,34,205,97,Shah- i- Kot Valley Q16205112,2002.0,False
4,4,71717,Second Intifada,military conflict,193420,680777,261,784,2005,Palestinian Authority|Israel,2005.0,False
5,5,140792,First Chechen War,military conflict,63641,985456,66,308,3000,Chechen Republic Q5187|Republic of Ingushe...,1996.0,False
6,6,171392,Second Chechen War,military conflict,134225,1385837,167,634,7321,Chechen Republic Q5187,2009.0,False
7,7,182821,Siege of Sarajevo,military conflict,78135,1060559,85,365,6137,Sarajevo Q11194,1996.0,False
8,8,201936,2003 invasion of Iraq,military conflict,250155,2953828,290,886,551,Iraq Q796,2003.0,False
9,9,202304,Protests against the Iraq War,civil conflict,85504,134088,111,317,0,Global,2002.0,False


#### scegliere quello giusto

In [6]:
articles_to_treat = articles[(articles['n_conflict']<=607)]
#articles_to_treat = articles[(articles['n_conflict']>607) & (articles['n_conflict']<=1214)]
#articles_to_treat = articles[(articles['n_conflict']>1214)]

In [7]:
def get_wiki_military_conflict_deaths(row):
    idPage = row['id']
    page = wptools.page(pageid=idPage)
    
    try:
        page.get_parse()
    except:
        return 0
    
    deaths = None
    
    try: 
        page.get_wikidata()
        deaths = page.data['wikidata']['number of deaths (P1120)']['amount']
        return deaths
    except:
        deaths = None
        
    try:
        deaths = page.data['infobox']['casualties3']
        return deaths
    except:
        deaths = None
        
    
    deaths2 = None
    
    try:
        deaths = page.data['infobox']['casualties1']
    except:
        return None
        
    try:
        deaths2 = page.data['infobox']['casualties2']
        deaths  = deaths + " |caus2| " + deaths2
    except:
        deaths2 = None
        
    if deaths:
        return deaths
    else:
        return row['death']

In [8]:
%%capture
articles_to_treat['death'] = articles_to_treat.apply(get_wiki_military_conflict_deaths,axis=1)

Attenzione a salvare con il nome giusto :)

In [9]:
articles_to_treat.to_csv(DATA_DIR + 'data_from_wptools_1.csv')

## MERGE

In [28]:
new_articles = pd.read_csv(DATA_DIR + 'data_from_wptools_3.csv', index_col=0)

In [29]:
new_articles = new_articles.set_index('id')

In [30]:
new_articles = new_articles.drop(['n_conflict'],axis=1)

In [31]:
new_articles

,title,categories,article_lenght,views,refs_count,link_count,death,locations,end_date_clean,ongoing
id,,,,,,,,,,
45560460,Eastern al-Hasakah offensive,military conflict,38855,30496,71,102,287–400 Assyrian civilians kidnapped (24 relea...,Al- Hasakah Governorate Q233914,2015.0,False
45619584,Salahuddin campaign,military conflict,34606,31169,74,92,"46 civilians killed <br>28,000 civilians displ...",Saladin Governorate Q190131,2016.0,False
45666688,Operation Rainbow (Syrian Civil War),military conflict,37816,38471,98,61,540+ killed |caus2| 270+ killed,Aleppo Governorate Q214064,2015.0,False
45683170,2015–16 protests in Brazil,civil conflict,40702,78927,61,153,NaN,nowrap Brazil 160 cities in 26 states an...,2016.0,False
45716178,Battle of Sirte (2015),military conflict,8919,49211,16,32,10 killed |caus2| 41 killed,Sirte Q162413,2015.0,False
45716539,Battle of Aden Airport,military conflict,10526,28742,14,55,"13 killed, 63 wounded",Aden Q131694,2015.0,False
46179869,Battle of Aden (2015),military conflict,42611,10348,85,115,858 civilians killed (256 by disease; pro-Hadi...,Aden Q131694,2015.0,False
46218259,Battle of Idlib (2015),military conflict,36787,12006,75,67,"11 civilians killed, 15 prisoners executed by ...",Idlib Q504657,2015.0,False
46222132,Battle of Bosra (2015),military conflict,8455,28941,12,27,4 civilians killed,Bosra Q272680,2015.0,False


In [34]:
old_articles.update(new_articles)

In [38]:
old_articles.describe()

,article_lenght,views,refs_count,link_count,end_date_clean
count,17459.000000,1.745900e+04,17459.000000,17459.000000,16848.000000
mean,18878.777536,1.137520e+05,20.878916,78.811902,1747.511871
std,27955.171439,6.499852e+05,44.011206,88.279906,403.200965
min,526.000000,4.100000e+01,0.000000,1.000000,19.000000
25%,5029.000000,5.606500e+03,2.000000,31.000000,1739.000000
50%,9633.000000,1.486200e+04,7.000000,51.000000,1912.000000
75%,20407.500000,4.779800e+04,20.000000,90.000000,1975.000000
max,636363.000000,3.383597e+07,1000.000000,1456.000000,2018.000000


In [54]:
#articles = articles.reset_index()

In [58]:
#articles['id'] = articles.id.astype(int)

In [60]:
#articles = articles.set_index('id')

In [67]:
#articles = articles.drop(['index'],axis=1)

ok

In [39]:
old_articles.to_csv(DATA_DIR + 'data_processed_before_death_cleaning.csv')